In [1]:
import warnings
warnings.filterwarnings(action="ignore")
import datetime
import time
import pandas as pd
import numpy as np
import xgboost as xgb
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
import optuna
from optuna.samplers import TPESampler
from optuna.pruners import HyperbandPruner

In [2]:
RANDOM_SEED = 99

In [3]:
train_df = pd.read_parquet('./data/train_v2_2_nn.parquet').drop(columns=["V_WIND"])
train_null_df = pd.read_parquet('./data/train_v2_2_n.parquet').drop(columns=["U_WIND", "V_WIND"])
test_df = pd.read_parquet('./data/test_v2_2.parquet').drop(columns=["U_WIND", "V_WIND"])

In [4]:
train_x = train_df.drop(columns=["U_WIND"])
train_y = train_df["U_WIND"]

In [4]:
def create_model(param):
    return xgb.XGBRegressor(
        random_state=RANDOM_SEED,
        tree_method="gpu_hist",
        eval_metric="rmse",
        early_stopping_rounds=100,
        **param
    )

In [14]:
def train_model(param, X, y, X_test, trial=None, test=False):
  skf = KFold(n_splits=3, shuffle=True, random_state=RANDOM_SEED)

  val_scores = []
  y_tests = []
  models = []

  for idx, (train_idx, val_idx) in enumerate(skf.split(X, y)):

    X_train, y_train = X.iloc[train_idx], y.iloc[train_idx]
    X_val, y_val = X.iloc[val_idx], y.iloc[val_idx]

    model = create_model(param)
    model.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_val, y_val)], verbose=1000)

    y_hat_val = model.predict(X_val)
    score = mean_squared_error(y_val.values, y_hat_val)
    val_scores.append(score)
    print(f'Fold: {idx+1}/3 mse score = {score:.5f}')

    if test:
      y_tests.append(model.predict(X_test))
      models.append(model)

    if trial:
      trial.report(score, idx)

      if trial.should_prune():
        raise optuna.TrialPruned()

  return val_scores, y_tests, models

In [15]:
def objective_xgb(trial):
  param = {
    "booster": trial.suggest_categorical("booster", ["gbtree", "dart"]),
    "lambda": trial.suggest_float("lambda", 1e-8, 1.0, log=True),
    "alpha": trial.suggest_float("alpha", 1e-8, 1.0, log=True),
    "subsample": trial.suggest_float("subsample", 0.1, 1.0),
    "colsample_bytree": trial.suggest_float("colsample_bytree", 0.1, 1.0),
    "n_estimators": trial.suggest_int("n_estimators", 100, 1000, step=100),
    "max_depth": trial.suggest_int("max_depth", 3, 9),
    "min_child_weight": trial.suggest_int("min_child_weight", 2, 10),
    "eta": trial.suggest_float("eta", 1e-8, 1.0, log=True),
    "gamma": trial.suggest_float("gamma", 1e-8, 1.0, log=True),
    "grow_policy": trial.suggest_categorical("grow_policy", ["depthwise", "lossguide"])
  }

  if param["booster"] == "dart":
    param['sample_type'] = trial.suggest_categorical('sample_type', ['uniform', 'weighted'])
    param['normalize_type'] = trial.suggest_categorical('normalize_type', ['tree', 'forest'])
    param['rate_drop'] = trial.suggest_float('rate_drop', 1e-8, 1.0, log=True)
    param['skip_drop'] = trial.suggest_float('skip_drop', 1e-8, 1.0, log=True)

  val_scores, y_tests, models = train_model(param, train_x, train_y, test_df, trial)

  return sum(val_scores) / len(val_scores)

In [16]:
study = optuna.create_study(
    sampler=TPESampler(seed=RANDOM_SEED),
    direction='minimize',
    study_name='xgb_tuning',
    pruner=HyperbandPruner(
      min_resource=1, max_resource=8, reduction_factor=3
    ),
)

study.optimize(objective_xgb, n_trials=50)

[I 2023-10-09 19:14:37,275] A new study created in memory with name: xgb_tuning


[0]	validation_0-rmse:3.46023	validation_1-rmse:3.45690
[299]	validation_0-rmse:3.46023	validation_1-rmse:3.45690
Fold: 1/3 rmse score = 11.95018
[0]	validation_0-rmse:3.46244	validation_1-rmse:3.45248
[299]	validation_0-rmse:3.46243	validation_1-rmse:3.45248
Fold: 2/3 rmse score = 11.91960
[0]	validation_0-rmse:3.45469	validation_1-rmse:3.46796
[299]	validation_0-rmse:3.45469	validation_1-rmse:3.46796


[I 2023-10-09 19:14:48,603] Trial 0 finished with value: 11.965502264735647 and parameters: {'booster': 'gbtree', 'lambda': 0.04017550942202233, 'alpha': 1.7847241527490656e-08, 'subsample': 0.8272449670283629, 'colsample_bytree': 0.6090556776494775, 'n_estimators': 300, 'max_depth': 3, 'min_child_weight': 10, 'eta': 1.1339812255777714e-08, 'gamma': 0.014399394343769443, 'grow_policy': 'depthwise'}. Best is trial 0 with value: 11.965502264735647.


Fold: 3/3 rmse score = 12.02673
[0]	validation_0-rmse:3.46023	validation_1-rmse:3.45690
[899]	validation_0-rmse:3.45960	validation_1-rmse:3.45630
Fold: 1/3 rmse score = 11.94600
[0]	validation_0-rmse:3.46243	validation_1-rmse:3.45248
[899]	validation_0-rmse:3.46182	validation_1-rmse:3.45186
Fold: 2/3 rmse score = 11.91535
[0]	validation_0-rmse:3.45469	validation_1-rmse:3.46796
[899]	validation_0-rmse:3.45408	validation_1-rmse:3.46734


[I 2023-10-09 19:47:16,063] Trial 1 finished with value: 11.961268186582119 and parameters: {'booster': 'dart', 'lambda': 1.4575798463318516e-05, 'alpha': 0.6189425995687715, 'subsample': 0.5719732438377161, 'colsample_bytree': 0.1842517839679773, 'n_estimators': 900, 'max_depth': 4, 'min_child_weight': 6, 'eta': 2.178476742104051e-06, 'gamma': 0.033817296080697445, 'grow_policy': 'depthwise', 'sample_type': 'uniform', 'normalize_type': 'tree', 'rate_drop': 1.4193450623251104e-07, 'skip_drop': 4.983900949448835e-07}. Best is trial 1 with value: 11.961268186582119.


...

Fold: 3/3 rmse score = 8.10362
[0]	validation_0-rmse:3.14676	validation_1-rmse:3.15776
[249]	validation_0-rmse:1.43798	validation_1-rmse:2.63684
Fold: 1/3 rmse score = 6.84296
[0]	validation_0-rmse:3.15785	validation_1-rmse:3.15290
[196]	validation_0-rmse:1.61332	validation_1-rmse:2.62752
Fold: 2/3 rmse score = 6.79896
[0]	validation_0-rmse:3.15462	validation_1-rmse:3.16588
[198]	validation_0-rmse:1.59510	validation_1-rmse:2.64453


[I 2023-10-09 23:48:24,977] Trial 48 finished with value: 6.8489854817684614 and parameters: {'booster': 'dart', 'lambda': 6.245080649354278e-05, 'alpha': 1.411695692872028e-08, 'subsample': 0.995283735646384, 'colsample_bytree': 0.7195388517886709, 'n_estimators': 1000, 'max_depth': 8, 'min_child_weight': 3, 'eta': 0.5125546374151384, 'gamma': 1.8147750606363866e-05, 'grow_policy': 'depthwise', 'sample_type': 'weighted', 'normalize_type': 'tree', 'rate_drop': 1.8006089654833396e-05, 'skip_drop': 1.930448249063886e-06}. Best is trial 45 with value: 5.962488010980658.


Fold: 3/3 rmse score = 6.90504
[0]	validation_0-rmse:3.33796	validation_1-rmse:3.33910
[999]	validation_0-rmse:1.34112	validation_1-rmse:2.45680
Fold: 1/3 rmse score = 6.03420
[0]	validation_0-rmse:3.34407	validation_1-rmse:3.33534
[999]	validation_0-rmse:1.33359	validation_1-rmse:2.43993
Fold: 2/3 rmse score = 5.95298
[0]	validation_0-rmse:3.33753	validation_1-rmse:3.35021
[999]	validation_0-rmse:1.34875	validation_1-rmse:2.43985


[I 2023-10-10 00:29:46,293] Trial 49 finished with value: 5.980015642325394 and parameters: {'booster': 'dart', 'lambda': 0.0026467279365743053, 'alpha': 1.0648149016043716e-07, 'subsample': 0.9558810566556822, 'colsample_bytree': 0.6990757999761278, 'n_estimators': 1000, 'max_depth': 8, 'min_child_weight': 5, 'eta': 0.16652538854459067, 'gamma': 1.1418655594749022e-05, 'grow_policy': 'depthwise', 'sample_type': 'weighted', 'normalize_type': 'tree', 'rate_drop': 0.0010395081599285753, 'skip_drop': 2.4105027920273678e-05}. Best is trial 45 with value: 5.962488010980658.


Fold: 3/3 rmse score = 5.95287


In [17]:
trial = study.best_trial

In [18]:
print(trial.params)

{'booster': 'dart', 'lambda': 0.002578975588016742, 'alpha': 1.3244432301036111e-08, 'subsample': 0.9983456796567938, 'colsample_bytree': 0.7040062969819831, 'n_estimators': 1000, 'max_depth': 9, 'min_child_weight': 6, 'eta': 0.16044236257401082, 'gamma': 4.4590714286410406e-05, 'grow_policy': 'depthwise', 'sample_type': 'weighted', 'normalize_type': 'tree', 'rate_drop': 9.776329107921942e-05, 'skip_drop': 4.429240963896771e-06}


In [5]:
best_param = {
  'booster': 'dart',
  'lambda': 0.002578975588016742,
  'alpha': 1.3244432301036111e-08,
  'subsample': 0.9983456796567938,
  'colsample_bytree': 0.7040062969819831,
  'n_estimators': 1000,
  'max_depth': 9,
  'min_child_weight': 6,
  'eta': 0.16044236257401082,
  'gamma': 4.4590714286410406e-05,
  'grow_policy': 'depthwise',
  'sample_type': 'weighted',
  'normalize_type': 'tree',
  'rate_drop': 9.776329107921942e-05,
  'skip_drop': 4.429240963896771e-06
}

# val_scores, y_tests, models = train_model(best_param, train_x, train_y, train_null_df, test=True)

In [24]:
for idx, model in enumerate(models):
  file_name = f"./models/uwind_model_{idx}.model"
  model.save_model(file_name)

In [6]:
models = []
for i in range(3):
  model = create_model(best_param)
  model.load_model(f"models/uwind_model_{i}.model")
  models.append(model)

In [7]:
test_winds = []
for model in models:
  test_winds.append(model.predict(train_null_df))

In [8]:
train_null_df["U_WIND"] = np.mean(test_winds, axis=0)

In [9]:
assert sorted(train_df.columns) == sorted(train_null_df.columns)

In [10]:
result = pd.concat([train_df, train_null_df], axis=0).sort_index()

In [11]:
result.head(10)

,ARI_CO_BR,ARI_CO_CA,ARI_CO_CL,ARI_CO_CN,ARI_CO_FI,ARI_CO_ID,ARI_CO_IN,ARI_CO_JP,ARI_CO_KR,ARI_CO_LV,...,SHIP_TYPE_CATEGORY_Tanker,U_WIND,WITlt0,WTI,day,hour,minute,month,weekday,year
0,0,0,0,1,0,0,0,0,0,0,...,0,0.140000,0,-1.081668,15,4,3,10,3,2020
1,0,0,0,1,0,0,0,0,0,0,...,0,-3.160000,0,-0.188614,17,2,55,9,1,2019
2,0,0,0,1,0,0,0,0,0,0,...,0,0.000000,0,-0.305226,23,6,43,2,5,2019
3,0,0,0,0,0,0,0,1,0,0,...,0,4.340000,0,-1.074380,18,22,6,9,4,2020
4,0,0,0,0,0,0,0,0,0,0,...,0,1.920000,0,1.209272,13,12,57,8,5,2022
5,0,0,0,1,0,0,0,0,0,0,...,0,-0.600093,0,-0.840670,8,14,24,9,1,2015
6,0,0,0,0,0,0,0,1,0,0,...,0,-1.910000,0,-0.507840,25,21,34,1,0,2021
7,0,0,0,0,0,0,0,1,0,0,...,0,0.000000,0,-0.459737,18,3,48,6,1,2019
8,0,0,0,0,0,0,0,1,0,0,...,0,0.195547,0,-0.488404,27,6,23,1,4,2017
9,0,0,0,0,0,0,0,0,0,0,...,0,1.886073,0,0.008168,3,7,8,2,5,2018


In [12]:
result.to_parquet("./data/train_v2_2u.parquet")

In [13]:
test_winds = []
for model in models:
  test_winds.append(model.predict(test_df))

In [14]:
test_df["U_WIND"] = np.mean(test_winds, axis=0)

In [15]:
test_df.to_parquet("./data/test_v2_2u.parquet")